In [32]:
%pip install torch 

Note: you may need to restart the kernel to use updated packages.


In [33]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import recall_score, precision_score, f1_score

import torch 
from torch import nn
from torch import optim

import datetime

In [35]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import ngrams
from nltk import word_tokenize, pos_tag
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys

In [36]:
import nltk

nltk.download("wordnet")
nltk.download("punkt")
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\agupt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\agupt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\agupt\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agupt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
# Load the training and validation data

train = pd.read_csv('../public_data/train/track_a/eng.csv')
val = pd.read_csv('../public_data/dev/track_a/eng_a.csv')

train.head()

,id,text,Anger,Fear,Joy,Sadness,Surprise
0,eng_train_track_a_00001,But not very happy.,0,0,1,1,0
1,eng_train_track_a_00002,Well she's not gon na last the whole song like...,0,0,1,0,0
2,eng_train_track_a_00003,She sat at her Papa's recliner sofa only to mo...,0,0,0,0,0
3,eng_train_track_a_00004,"Yes, the Oklahoma city bombing.",1,1,0,1,1
4,eng_train_track_a_00005,They were dancing to Bolero.,0,0,1,0,0


In [38]:
## Add Tokeniser search
from transformers import BertTokenizer

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# POS Tagging
import spacy
nlp = spacy.load("en_core_web_sm")

# SPACY TOKENISER SETUP 
# import spacy
# nlp = spacy.load("en_core_web_sm")

# NLTK TREEBANK TOKENISER SETUP
# from nltk.tokenize import TreebankWordTokenizer

# NLTK WORDPUNCT TOKENISER SETUP 
# from nltk.tokenize import WordPunctTokenizer


# Tokenize and encode the text

def pre_process(text, config):
    """ 
    Performs Different preprocessing operations.

    Parameters:
    text (string): passes a line of text (assume sentence segmentation has already been done)

    Returns:
    List[string]: Should return a list of tokens.
    """

    def separate_punctuation(text):
        text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
        text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\1 \2", text)
        return text

    def remove_punctuation(text):
        text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1", text)
        text = re.sub(r"([.,;:!?'\"“\(\)])(\w)", r"\2", text)
        return text
        
    def tokenize_text(text):
        # Original tokeniser:
        # tokens = re.split(r"\s+",text)
        # tokens = [t.lower() for t in tokens]
        
        # BERT TOKENISER SETUP
        # Encode text returns a BatchEncoding object
        encoded_input = tokenizer(text, return_tensors='pt', add_special_tokens=True)
        # Convert token ids to words
        tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
        return tokens

        # SPACY TOKENISER SETUP
        # doc = nlp(text)
        # # Collect only the text of each token in the document
        # tokens = [token.text for token in doc]

        # NLTK TREEBANKWORD TOKENIZER 
        # tokenizer = TreebankWordTokenizer()
        # tokens = tokenizer.tokenize(text)

        # NLTK WORDPUNCT TOKENIZER
        # tokenizer = WordPunctTokenizer()
        # tokens = tokenizer.tokenize(text)

        # Return the list of tokens
        # return tokens

    def apply_stemming(tokens):
        stemmer = PorterStemmer()
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        return stemmed_tokens

    def apply_lemmatization(tokens):
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return lemmatized_tokens

    def generate_ngrams_from_tokens(tokens, n):
        return list(ngrams(tokens, n))

    # Separate Punctuation otherwise Remove it
    
    if config["sep_pn"] and not config["rm_pn"]:
        text = separate_punctuation(text)

    if config["rm_pn"] and not config["sep_pn"]:
        text = remove_punctuation(text)
    
    # tokenize text
    
    tokens = tokenize_text(text)

    # POS Tagging
    # Reconstruct the full text from subwords, handling special tokens
    reconstructed_text = tokenizer.convert_tokens_to_string(tokens)
    # Process the reconstructed text with spaCy
    doc = nlp(reconstructed_text)

    # POS tagging
    for token in doc:
        print(token.text, token.pos_)

    # Apply Lemmatization or Stemming

    if config["apply_stemming"]:
        tokens = apply_stemming(tokens)
    if config["apply_lemmatization"]:
        tokens = apply_lemmatization(tokens)

    # Generate bigrams, trigrams and quadgrams
    if config["add_bigrams"]:
        bigrams = generate_ngrams_from_tokens(tokens, 2)
        bg = [i + " " + j for (i,j) in bigrams]
        tokens += bg

    # Remove Stop words
    
    if config["rm_sw"]:
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if w not in stop_words]

    # stop_words = set(stopwords.words('english'))
    # tokens = [w for w in tokens if w not in stop_words]

    return " ".join(tokens)

In [39]:
# import itertools

# def grid_configurations():
#     options = [
#         "sep_pn", "rm_pn", "apply_lemmatization", "apply_stemming", "add_bigrams", "rm_sw"
#     ]
    
#     # Generate all combinations of True/False for each option
#     combinations = itertools.product([True, False], repeat=len(options))
    
#     configurations = []
    
#     # Create a dictionary for each combination
#     for combo in combinations:
#         config = {options[i]: combo[i] for i in range(len(options))}
#         configurations.append(config)
    
#     return configurations

# # Example usage
# results = []
# configs = grid_configurations()
# for _i, config in enumerate(configs):
#     vectorizer = CountVectorizer()
#     X_train = vectorizer.fit_transform([pre_process(i, config) for i in train["text"]]).toarray()
#     X_val = vectorizer.transform(val['text'].str.lower()).toarray()

#     emotions = ['Joy','Sadness','Surprise','Fear','Anger']
#     y_train = train[emotions].values
#     y_val = val[emotions].values

#     X_train_t = torch.Tensor(X_train)
#     y_train_t = torch.Tensor(y_train)

#     X_val_t = torch.Tensor(X_val)
#     y_val_t = torch.Tensor(y_val)

#     model = nn.Sequential(
#           nn.Linear(X_train.shape[1], 100),
#           nn.ReLU(),
#           nn.Dropout(0.3),
#           nn.Linear(100, y_train.shape[1])
#         )
    
#     criterion = nn.BCEWithLogitsLoss()
#     optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)
#     loss = None
#     # Train for a set number of epochs
#     for epoch in range(200):
#         optimizer.zero_grad()
#         output = model(X_train_t)
#         loss = criterion(output, y_train_t)
#         loss.backward()
#         optimizer.step()
#     results.append((round(loss.item(),3), _i))
#     print(_i, round(loss.item(),3), config)
# print("MIN LOSS: ", min(results))

In [40]:
# [(i, pre_process(i, {'sep_pn': True, 'rm_pn': False, 'apply_lemmatization': True, 'apply_stemming': True, 'add_bigrams': True, 'rm_sw': False})) for i in train["text"]]

In [41]:
# ## Add Tokeniser search
# from transformers import BertTokenizer

# # Load pre-trained BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenize and encode the text

# tokens = tokenizer("Example text for sentiment analysis", return_tensors="pt")

# `inputs` now contains input IDs and attention masks that can be fed into a BERT model

In [42]:
# import spacy
# nlp = spacy.load("en_core_web_sm")
# doc = nlp("Your sample text goes here.")
# tokens = [token.text for token in doc]

# from nltk.tokenize import TweetTokenizer
# tokenizer = TweetTokenizer()
# tokens = tokenizer.tokenize("Your sample tweet goes here :) #exciting")

In [43]:
vectorizer = CountVectorizer()

# Use TF-IDF Vectoriser instead

# from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object
# vectorizer = TfidfVectorizer()

val_features = [pre_process(i, config) for i in val["text"]]

# Optimal config #18 
config = {'sep_pn': True, 'rm_pn': False, 'apply_lemmatization': True, 'apply_stemming': True, 'add_bigrams': True, 'rm_sw': False}

val_features = [pre_process(i, config) for i in val["text"]]

X_train = vectorizer.fit_transform([pre_process(i, config) for i in train["text"]]).toarray()
X_val = vectorizer.transform(val_features).toarray()

emotions = ['Joy','Sadness','Surprise','Fear','Anger']
y_train = train[emotions].values
y_val = val[emotions].values

print(X_val.shape)
print(X_train.shape)

[ X
CLS X
] X
my PRON
mouth NOUN
fell VERB
open ADJ
` PUNCT
` PUNCT
no INTJ
, PUNCT
no INTJ
, PUNCT
no INTJ
. PUNCT
. PUNCT
. PUNCT
i PRON
. PUNCT
. PUNCT
[ X
SEP PROPN
] PUNCT
[ X
CLS X
] X
you PRON
can AUX
barely ADV
make VERB
out ADP
your PRON
daughter NOUN
' PUNCT
s PART
pale ADJ
form NOUN
in ADP
the DET
darkness NOUN
of ADP
your PRON
room NOUN
. PUNCT
[ X
SEP PROPN
] PUNCT
[ X
CLS X
] X
but CCONJ
after ADP
blinking VERB
my PRON
eyes NOUN
for ADP
a DET
few ADJ
times NOUN
lepas VERB
tu PROPN
tampar NOUN
- PUNCT
tampar NOUN
muka PROPN
sikit PROPN
, PUNCT
memang PROPN
sah PROPN
la PROPN
yang PROPN
penghantar PROPN
itu PROPN
hanis PROPN
zalikha PROPN
. PUNCT
[ X
SEP PROPN
] PUNCT
[ X
CLS X
] X
slowly ADV
rising VERB
to ADP
my PRON
feet NOUN
i PRON
came VERB
to ADP
the DET
conclusion NOUN
that SCONJ
sometimes ADV
being AUX
alone ADV
meant VERB
surviving VERB
for ADP
myself PRON
. PUNCT
[ X
SEP PROPN
] PUNCT
[ X
CLS X
] X
i PRON
noticed VERB
this DET
months NOUN
after ADP
moving VERB
in 

In [44]:
# X_train_t = torch.Tensor(X_train)
# y_train_t = torch.Tensor(y_train)

# X_val_t = torch.Tensor(X_val)
# y_val_t = torch.Tensor(y_val)

In [45]:
print(f'Shape of X: {X_train.shape}')
print(f'Shape of y: {y_train.shape}')
print(f'Number of positives per emotion class:')
_ = [print(f' - {e}: {v} ({round(100*v/len(y_train))}%)') for e,v in zip(emotions, y_train.sum(axis=0))]

Shape of X: (2768, 4072)
Shape of y: (2768, 5)
Number of positives per emotion class:
 - Joy: 674 (24%)
 - Sadness: 878 (32%)
 - Surprise: 839 (30%)
 - Fear: 1611 (58%)
 - Anger: 333 (12%)


In [46]:
# Use POS Tagging

# Function to extract POS tags
def extract_pos_tags(texts):
    tagged_texts = []
    for text in texts:
        doc = nlp(text)
        tagged_texts.append(" ".join([token.pos_ for token in doc]))
    return tagged_texts

# print(train["text"])

# Extract POS tags
train_pos_tags = extract_pos_tags(train["text"])
print(train_pos_tags)

from sklearn.preprocessing import OneHotEncoder
import numpy as np
# Instantiate the encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# try:
#     # Split each string into a list of tags and encode
#     pos_encoded = encoder.fit_transform([tags.split() for tags in pos_tags])
#     print("Encoding successful. Shape:", pos_encoded.shape)
# except Exception as e:
#     print("Error during encoding:", e)


# empty_or_irregular = [tags for tags in pos_tags if not tags or len(tags.split()) == 0]
# print("Empty or irregular entries:", empty_or_irregular[:10])

# Find the maximum number of tags in any sentence
train_max_length = max(len(tags.split()) for tags in train_pos_tags)
print(train_max_length)


# Pad the shorter lists with 'PAD' to ensure uniform length
train_padded_pos_tags = [tags.split() + ['PAD'] * (train_max_length - len(tags.split())) for tags in train_pos_tags]
print(train_padded_pos_tags)
unique_tags = set(tag for sentence in train_padded_pos_tags for tag in sentence)
print(unique_tags)
# Now encode the padded lists
try:
    train_pos_encoded = encoder.fit_transform(train_padded_pos_tags)
    print("Encoding successful. Shape:", train_pos_encoded.shape)
except Exception as e:
    print("Error during encoding after padding:", e)


# Fit and transform the POS tags
# pos_encoded = encoder.fit_transform([tags.split() for tags in pos_tags])
# pos_encoded_dense = pos_encoded.toarray()  # Convert sparse matrix to dense

# assert X_train.shape[0] == pos_encoded.shape[0], "Mismatch in number of rows between X and POS encoded features."

combined_features = np.concatenate((X_train, train_pos_encoded), axis=1)

# POS Tagging for Validation set 

val_pos_tags = extract_pos_tags(val["text"])
print(val_pos_tags)
# train_tags = set(tag for tags in train_pos_tags for tag in tags.split())
# val_tags = set(tag for tags in val_pos_tags for tag in tags.split())

# print("Tags in training set but not in validation:", train_tags - val_tags)
# print("Tags in validation set but not in training:", val_tags - train_tags)

# try:
#     # Split each string into a list of tags and encode
#     pos_encoded = encoder.fit_transform([tags.split() for tags in pos_tags])
#     print("Encoding successful. Shape:", pos_encoded.shape)
# except Exception as e:
#     print("Error during encoding:", e)


# empty_or_irregular = [tags for tags in pos_tags if not tags or len(tags.split()) == 0]
# print("Empty or irregular entries:", empty_or_irregular[:10])

# Find the maximum number of tags in any sentence
# val_max_length = max(len(tags.split()) for tags in val_pos_tags)
# print(val_max_length)

# Pad the shorter lists with 'PAD' to ensure uniform length
val_padded_pos_tags = [tags.split() + ['PAD'] * (train_max_length - len(tags.split())) for tags in val_pos_tags]
print(val_padded_pos_tags)
# Now encode the padded lists
# try:
val_pos_encoded = encoder.transform(val_padded_pos_tags) ## Same encoder that has already been trained
print("Encoding successful. Shape:", val_pos_encoded.shape)
# except Exception as e:
#     print("Error during encoding:", e)


# Fit and transform the POS tags
# pos_encoded = encoder.fit_transform([tags.split() for tags in pos_tags])
# pos_encoded_dense = pos_encoded.toarray()  # Convert sparse matrix to dense

# assert X_train.shape[0] == pos_encoded.shape[0], "Mismatch in number of rows between X and POS encoded features."

validation_combined_features = np.concatenate((X_val, val_pos_encoded), axis=1)

print(combined_features.shape)
print(validation_combined_features.shape)

# Now pos_encoded contains a one-hot encoded numpy array of POS tags
# Assuming 'other_features' is your main feature matrix derived from your text (e.g., TF-IDF)
# Make sure both arrays have the same number of rows
# combined_features = np.concatenate((X_train, pos_encoded), axis=1)

['CCONJ PART ADV ADJ PUNCT', 'INTJ PRON AUX PART VERB PART VERB DET ADJ NOUN ADP PRON PUNCT CCONJ SCONJ PRON AUX ADP PRON CCONJ DET NOUN AUX PART VERB ADP PRON NOUN ADV ADV PUNCT PRON VERB PRON NOUN PART VERB ADP ADP DET NOUN CCONJ VERB PRON NOUN PUNCT', 'PRON VERB ADP PRON PROPN PART NOUN NOUN ADV PART VERB ADV ADP PRON CCONJ VERB VERB ADP PRON NOUN PUNCT', 'INTJ PUNCT DET PROPN NOUN NOUN PUNCT', 'PRON AUX VERB ADP PROPN PUNCT', 'ADV VERB NOUN ADP PRON PUNCT ADV PUNCT', 'CCONJ PRON AUX VERB PUNCT PRON NOUN VERB SCONJ PRON AUX ADJ PART VERB ADP ADP PRON NOUN PUNCT PRON VERB DET VERB NOUN CCONJ NOUN PART VERB PRON NOUN PUNCT ADV ADP DET ADJ NOUN PUNCT ADV VERB PRON ADV SYM NOUN NUM NOUN PROPN PART PROPN PROPN PUNCT NUM NOUN ADV ADJ NOUN PUNCT ADJ NOUN ADP NOUN ADP NOUN PUNCT', 'PRON VERB DET NOUN ADP PROPN ADV ADP VERB NOUN ADP DET NOUN SCONJ ADP DET ADJ NOUN DET NOUN VERB PUNCT', 'ADV PRON NOUN VERB ADP DET NOUN PUNCT CCONJ DET NOUN AUX ADP DET ADJ NOUN PUNCT', 'PRON VERB ADP DET NOUN 

In [47]:
model = nn.Sequential(
          nn.Linear(combined_features.shape[1], 100),
          nn.BatchNorm1d(100),  # Batch normalization for 100 features
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(100, y_train.shape[1])
        )

In [48]:
# model = nn.Sequential(
#     nn.Linear(combined_features.shape[1], 100),
#     nn.BatchNorm1d(100),  # Batch normalization for 100 features
#     nn.ReLU(),
#     nn.Dropout(0.3),
#     nn.Linear(100, 50),   # Additional intermediate layer
#     nn.BatchNorm1d(50),   # Batch normalization for 50 features
#     nn.ReLU(),
#     nn.Linear(50, y_train.shape[1]),
#     nn.BatchNorm1d(y_train.shape[1]),  # Optionally add BatchNorm here if suitable
#     nn.ReLU()  # Final activation if needed (depends on the output requirement)
# )

In [49]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm 

# Add early stopping" TBC 
# best_loss = float('inf')
# patience = 20  # Number of epochs to wait for improvement before stopping
# patience_counter = 0  # Tracks how many epochs have passed without improvement


# Convert to torch tensors
features_tensor = torch.tensor(combined_features, dtype=torch.float32)
labels_tensor = torch.tensor(y_train, dtype=torch.float32)

# Create dataset and data loader
dataset = TensorDataset(features_tensor, labels_tensor)
data_loader = DataLoader(dataset, batch_size=10, shuffle=True)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss() # Use BCE with Logit Loss because it is multi-hot
optimizer = optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-2) ## Lower the learning rate from 1e-1 to 1e-2 to try to decrease the fluctuation in loss

# Training loop
for epoch in tqdm(range(1000), desc="Training Loop:"):
    for features, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')
    if epoch == 1000:
        print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

    # print(f'Epoch {epoch+1}, Loss: {loss.item()}')

"""Previous results:

Training Loop::   0%|          | 1/1000 [00:00<08:14,  2.02it/s]
Epoch 0: Loss: 0.432
Training Loop::  10%|█         | 101/1000 [01:54<19:53,  1.33s/it]
Epoch 100: Loss: 0.194
Training Loop::  20%|██        | 201/1000 [04:53<42:45,  3.21s/it]
Epoch 200: Loss: 0.166
Training Loop::  30%|███       | 301/1000 [06:41<07:41,  1.51it/s]
Epoch 300: Loss: 0.155
Training Loop::  40%|████      | 401/1000 [08:21<08:29,  1.18it/s]
Epoch 400: Loss: 0.196
Training Loop::  50%|█████     | 501/1000 [09:52<07:15,  1.15it/s]
Epoch 500: Loss: 0.153
Training Loop::  60%|██████    | 601/1000 [11:55<07:03,  1.06s/it]
Epoch 600: Loss: 0.245
Training Loop::  70%|███████   | 701/1000 [13:46<11:18,  2.27s/it]
Epoch 700: Loss: 0.161
Training Loop::  80%|████████  | 801/1000 [15:22<03:38,  1.10s/it]
Epoch 800: Loss: 0.215
Training Loop::  90%|█████████ | 901/1000 [16:29<01:01,  1.62it/s]
Epoch 900: Loss: 0.152
Training Loop:: 100%|██████████| 1000/1000 [19:10<00:00,  1.15s/it]

"""

Training Loop::   0%|          | 1/1000 [00:04<1:08:12,  4.10s/it]

Epoch 0: Loss: 0.582


Training Loop::  10%|█         | 101/1000 [03:38<24:03,  1.61s/it]

Epoch 100: Loss: 0.166


Training Loop::  20%|██        | 201/1000 [06:45<31:29,  2.36s/it]

...

Epoch 800: Loss: 0.132


Training Loop::  90%|█████████ | 901/1000 [33:38<03:18,  2.01s/it]

Epoch 900: Loss: 0.233


Training Loop:: 100%|██████████| 1000/1000 [36:35<00:00,  2.20s/it]


'Previous results:\n\nTraining Loop::   0%|          | 1/1000 [00:00<08:14,  2.02it/s]\nEpoch 0: Loss: 0.432\nTraining Loop::  10%|█         | 101/1000 [01:54<19:53,  1.33s/it]\nEpoch 100: Loss: 0.194\nTraining Loop::  20%|██        | 201/1000 [04:53<42:45,  3.21s/it]\nEpoch 200: Loss: 0.166\nTraining Loop::  30%|███       | 301/1000 [06:41<07:41,  1.51it/s]\nEpoch 300: Loss: 0.155\nTraining Loop::  40%|████      | 401/1000 [08:21<08:29,  1.18it/s]\nEpoch 400: Loss: 0.196\nTraining Loop::  50%|█████     | 501/1000 [09:52<07:15,  1.15it/s]\nEpoch 500: Loss: 0.153\nTraining Loop::  60%|██████    | 601/1000 [11:55<07:03,  1.06s/it]\nEpoch 600: Loss: 0.245\nTraining Loop::  70%|███████   | 701/1000 [13:46<11:18,  2.27s/it]\nEpoch 700: Loss: 0.161\nTraining Loop::  80%|████████  | 801/1000 [15:22<03:38,  1.10s/it]\nEpoch 800: Loss: 0.215\nTraining Loop::  90%|█████████ | 901/1000 [16:29<01:01,  1.62it/s]\nEpoch 900: Loss: 0.152\nTraining Loop:: 100%|██████████| 1000/1000 [19:10<00:00,  1.15

In [50]:
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

In [51]:
# Train for a set number of epochs
# for epoch in range(1000):
#     optimizer.zero_grad()
#     output = model(X_train_t)
    # loss = criterion(output, y_train_t)
    # loss.backward()
    # optimizer.step()    
    # if epoch % 100 == 0:
    #     print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

In [52]:
def get_predictions(X_val, model, threshold=0.5):
    sig = nn.Sigmoid() 
    yhat = sig(model(X_val)).detach().numpy()
    y_pred = yhat > threshold
    
    return y_pred

In [53]:
# Check the shape of the input features for validation and model’s expected input
validation_features_tensor = torch.tensor(validation_combined_features, dtype=torch.float32)

print("Validation data shape:", validation_features_tensor.shape)
print("Expected input shape:", model[0].in_features)

Validation data shape: torch.Size([116, 5247])
Expected input shape: 5247


In [54]:
y_pred = get_predictions(validation_features_tensor, model, 0.45)
# print(y_pred)

# Create a DataFrame to save to CSV
val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])  # Adjust column names as per your features
# val_data_with_pred['True_Label'] = y_test
# val_data_with_pred['Predictions'] = dummy_predictions

val_data_with_pred = val_data_with_pred.astype(int)

val_data_with_pred['id'] = val['id']

val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Save to CSV
current_time = datetime.datetime.now()
formatted_time = current_time.strftime('%Y-%m-%d_%H_%M_%S')

val_data_with_pred.to_csv(f'../results/pred_eng_a_{formatted_time}.csv', index=False)

print(val_data_with_pred)

                        id  Anger  Fear  Joy  Sadness  Surprise
0    eng_dev_track_a_00001      0     1    0        0         0
1    eng_dev_track_a_00002      0     0    0        1         0
2    eng_dev_track_a_00003      1     0    1        0         0
3    eng_dev_track_a_00004      1     0    0        0         0
4    eng_dev_track_a_00005      0     0    0        1         0
..                     ...    ...   ...  ...      ...       ...
111  eng_dev_track_a_00112      0     0    1        1         0
112  eng_dev_track_a_00113      0     1    1        1         0
113  eng_dev_track_a_00114      0     0    0        1         0
114  eng_dev_track_a_00115      0     0    0        0         0
115  eng_dev_track_a_00116      0     0    0        0         0

[116 rows x 6 columns]


In [55]:
# def evaluate(y_val, y_pred):
#     for average in ['micro', 'macro']:
#         recall = recall_score(y_val, y_pred, average=average, zero_division=0)
#         precision = precision_score(y_val, y_pred, average=average, zero_division=0)
#         f1 = f1_score(y_val, y_pred, average=average, zero_division=0)
    
#         print(f'{average.upper()} recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}')

In [56]:
# evaluate(y_val, y_pred) EVALS WON'T WORK HERE 

In [57]:
# def evaluate_per_class(y_val, y_pred):
#     for i, emotion in enumerate(emotions):
#         print(f'*** {emotion} ***')
    
#         recall = recall_score(y_val[:,i], y_pred[:,i], zero_division=0)
#         precision = precision_score(y_val[:,i], y_pred[:,i], zero_division=0)
#         f1 = f1_score(y_val[:,i], y_pred[:,i], zero_division=0)
        
#         print(f'recall: {round(recall, 4)}, precision: {round(precision, 4)}, f1: {round(f1, 4)}\n')

In [58]:
# evaluate_per_class(y_val, y_pred) EVALS WON'T WORK HERE 

In [59]:
# weights = y_train.sum(axis=0)/y_train.sum()
# weights = max(weights)/weights

In [60]:
# # # Define model 
# # model = nn.Sequential(
# #           nn.Linear(X_train.shape[1], 100),
# #           nn.ReLU(),
# #           nn.Dropout(0.3),
# #           nn.Linear(100, y_train.shape[1])
# #         )

# # Define training parameters
# # criterion = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor(weights)) # <-- weights assigned to optimiser
# # optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(model.parameters(), lr=1e-1, weight_decay=1e-2)

# # # Train for a number of epochs
# # for epoch in range(1000):
# #     optimizer.zero_grad()
# #     output = model(X_train_t)
# #     loss = criterion(output, y_train_t)
# #     loss.backward()
# #     optimizer.step()
# #     if epoch % 100 == 0:
# #         print(f'Epoch {epoch}: Loss: {round(loss.item(),3)}')

# # # Get predictions
# # y_pred = get_predictions(X_val_t, model, 0.45)

# # # Create a DataFrame to save to CSV
# # val_data_with_pred = pd.DataFrame(y_pred, columns=['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])  # Adjust column names as per your features
# # # val_data_with_pred['True_Label'] = y_test
# # # val_data_with_pred['Predictions'] = dummy_predictions

# # val_data_with_pred = val_data_with_pred.astype(int)

# # val_data_with_pred['id'] = val['id']

# # val_data_with_pred = val_data_with_pred[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# # Save to CSV
# current_time = datetime.datetime.now()
# formatted_time = current_time.strftime('%Y-%m-%d_%H_%M_%S')

# val_data_with_pred.to_csv(f'../results/pred_eng_a_{formatted_time}.csv', index=False)

# # print(val_data_with_pred)

# # # Evaluate: EVALS WON'T WORK HERE 
# # # print('\n\nEVALUATION\n')
# # # evaluate(y_val, y_pred)

# # # print('\nPER CLASS BREAKDOWN\n')
# # # evaluate_per_class(y_val, y_pred)

In [61]:
# val_data_with_pred